# Exercise 1

In [ ]:
from random import random

# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network

For the hidden layer we create n_hidden neurons and each neuron in the hidden layer has n_inputs + 1 weights, one for each input column in a dataset and an additional one for the bias.

The output layer that connects to the hidden layer has n_outputs neurons, each with n_hidden + 1 weights. This means that each neuron in the output layer connects to (has a weight for) each neuron in the hidden layer.

# Exercise 2

In [ ]:
from random import seed

seed(1)
network = initialize_network(2, 1, 2)
for layer in network:
	print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


The hidden layer has one neuron with 2 input weights plus the bias and the output layer has 2 neurons, each with 1 weight plus the bias.

# Exercise 3

In [ ]:
# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation

Neuron activation is calculated as the weighted sum of the inputs : activation = sum(weight_i * input_i) + bias

The function assumes that the bias is the last weight in the list of weights.

# Exercise 4

In [ ]:
from math import exp

# Transfer neuron activation
def transfer(activation):
	return 1.0 / (1.0 + exp(-activation))

The sigmoid activation function looks like an S shape, it’s also called the logistic function. It can take any input value and produce a number between 0 and 1 on an S-curve. It is also a function of which we can easily calculate the derivative (slope) that we will need later when backpropagating error.

# Exercise 5

In [ ]:
# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs

A neuron’s output value is stored in the neuron with the name 'output'. We collect the outputs for a layer in an array named new_inputs that becomes the array inputs and is used as inputs for the following layer.

# Exercise 6

In [ ]:
seed(1)

# test forward propagation
network = initialize_network(2, 1, 2)
row = [0, 1]
output = forward_propagate(network, row)
print(output)

[0.6699713080575971, 0.7361939293022448]


We define our network inline with one hidden neuron that expects 2 input values and an output layer with two neurons.

Running the example propagates the input pattern [0, 1] and produces an output value that is printed. Because the output layer has two neurons, we get a list of two numbers as output.

# Exercise 7

In [ ]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)

We are using the sigmoid transfer function, the derivative of which can be calculated as follows: derivative = output * (1.0 - output)

# Exercise 8

In [ ]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

The error for a given output neuron can be calculated as follows: error = (expected - output) * transfer_derivative(output)

The error for a given hidden neuron can be calculated as follows: error = (weight_k * error_j) * transfer_derivative(output)

The error signal calculated for each neuron is stored with the name ‘delta’.

The layers of the network are iterated in reverse order, starting at the output and working backwards. This ensures that the neurons in the output layer have ‘delta’ values calculated first that neurons in the hidden layer can use in the subsequent iteration.

The error signal for neurons in the hidden layer is accumulated from neurons in the output layer where the hidden neuron number j is also the index of the neuron’s weight in the output layer neuron['weights'][j].

# Exercise 9

In [ ]:
seed(1)

# test backpropagation of error
network = initialize_network(2, 1, 2)
expected = [0, 1]
forward_propagate(network, expected)
backward_propagate_error(network, expected)
for layer in network:
	print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'output': 0.8335790831681538, 'delta': -0.0020469977629036075}]
[{'weights': [0.2550690257394217, 0.49543508709194095], 'output': 0.6699713080575971, 'delta': -0.1481371914045779}, {'weights': [0.4494910647887381, 0.651592972722763], 'output': 0.7361939293022448, 'delta': 0.05123441744823936}]


Running the example prints the network after the backpropagation of error is complete. Error values are calculated and stored in the neurons for the output layer and the hidden layer.